In [ ]:
folgendes uebertragen ins NB auf Arbeit:

In [ ]:
IDEE 2021-4-3: viell. check auf 1/omega nur auf Frequenzbereich bis zu bestimmter Grenzfrequenz (20 Hz, 50 Hz o.ae.) anwenden

In [ ]:
* fuer metric_hist3 (oder 2? Jedenfalls die Version, wo im Histogramm die einzelnen Fehlerarten verschiedene Farben haben) noch die Option anbieten, senkrechte Linien einzuzeichnen (unterschiedliche fuer jedes Diagramm), dann auf diese Weise die Resultate der GridSearch darstellen um zu sehen, wie gut die einzelnen Fehlerarten getrennt werden


In [ ]:
* Gridsearch:

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
# function to calculate the candidates for threshold values
# ... beschreiben
def get_candidates(x):
    t2=np.sort(x)                         # sort values
    gap = (t2[-1]-t2[0])/len(t2)          # aux. value to add elements at begin and end
    t3 = [t2[0]-gap, *t2, t2[-1]+gap]     # append elements at begin and end
    t4 = list(zip(t3[:-1], t3[1:]))       # aux. list for calculating the intermediate values
    return(np.unique([(v[1]+v[0])/2 for v in t4]))   # calculate the intermediate values 

In [14]:
# classificator
# TODO: verbessern, moeglichst durch einen ersetzen, der
#     * robuster ist (viell. mittels kde)
#     * schnelleren Suchalgorithmus hat
# fitcrit: criterion that should be used as optimization criterion for fit method 
class multi_threshold_classifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, ar__maxslope=np.arange(1000, 2000, 10), 
                 ar__asym=np.arange(0.01,0.03,0.001), 
                 ar__maxamp=np.arange(10000,20000,100),
                 fitcrit='hits'):
        
        """
        Called when initializing the classifier
        """
        self.ar__maxslope = ar__maxslope
        self.ar__asym = ar__asym
        self.ar__maxamp = ar__maxamp
        self.fitcrit = fitcrit

        
    # TODO: analytisch berechnen aehnlich wie bei lin.regr., formeln ableiten
    def fit(self, X, y):
        ym = [el==1 for el in y]
        # hier erstmal brut force, evtl. noch verbessern:
        # z.B. alle columns einzeln ordnen, zwischen Nachbarwerten interpolieren und nur diese Werte als
        # thresholds einsetzen, dann vielleicht auch viel einfacher die Werte rauszusuchen, wo z.B. die Anz. TP > Anz FP oder so
        if self.ar__maxslope is None:
            msls = get_candidates(X[:,0])
            # analog die anderen
        else:
            msls == self.ar__maxslope

        if self.ar__asym is None:
            asys = get_candidates(X[:,1])
            # analog die anderen
        else:
            asys == self.ar__asys

        if self.ar__maxamp is None:
            mams = get_candidates(X[:,2])
            # analog die anderen
        else:
            mams == self.ar__maxamp
            
        res = list()
        if fitcrit == 'hits':
            for msl in msls:
                for asy in asys:
                    for mam in mams:
                        y_pred = (X[:,0] >= msl) | (X[:,1]>=asy) | (X[:,2] <= mam)
                        res.append(msl, asy, mam, sum(y_pred==ym))
            df = pd.DataFrame.from_records(res, columns = ['maxslope', 'asym', 'maxamp', 'accur'])    
            im = df.loc[:, 'accur'].idxmax()
                    
            self.thres__maxslope = df.loc[im, 'maxslope']
            self.thres__asym = df.loc[im, 'asym']
            self.thres__maxamp == df.loc[im, 'maxamp']

        else:
            print('noch nicht implementiert')
            df = pd.DataFrame()
                
        return self

    
    
    #def _meaning(self, x):
    #    # returns True/False according to fitted classifier
    #    # notice underscore on the beginning
    #    y = (X[:,0] >= thres__maxslope) | (X[:,1]>=thres__asym) | (X[:,2] <= thres__maxamp)
    # 
    #    return( True if x >= self.treshold_ else False )
    #
    #def predict(self, X, y=None):
    #    try:
    #        getattr(self, "treshold_")
    #    except AttributeError:
    #        raise RuntimeError("You must train classifer before predicting data!")
    #    return([self._meaning(x) for x in X])


    """
    X must contain three columns (in the following order):
        1. maxslope-values
        2. asym-values
        3. maxamp-values
    """
    def predict(self, X):
        try:
            getattr(self, "thres__maxslope")
            getattr(self, "thres__asym")
            getattr(self, "thres__maxamp")
            
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        
        y = (X[:,0] >= thres__maxslope) | (X[:,1]>=thres__asym) | (X[:,2] <= thres__maxamp)
        return(y)
    
        
    def score(self, X, y):
        return(sum(y==self.predict(X)))

    
    def accuracy(self, y_test, y_pred):
        return(sum(y_test==y_pred)/len(y_test))
    

In [ ]:
Damit 

In [ ]:
from sklearn.cross_validation import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(df.loc[:, ['maxslope', 'asym', 'maxamp']].values, 3, test_size=0.5, random_state=0)
>>> len(sss)
3
>>> print(sss)       
StratifiedShuffleSplit(labels=[0 0 1 1], n_iter=3, ...)
>>> for train_index, test_index in sss:
...    print("TRAIN:", train_index, "TEST:", test_index)
...    X_train, X_test = X[train_index], X[test_index]
...    y_train, y_test = y[train_index], y[test_index]

In [11]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'ar__maxslope': linspace(2000, 10000,10),
                     'ar__asym': np.linspace(1e-5, 1e-4, 1e-5),
                     'ar__maxamp': np.linspace(1000,2000,10)}]

NameError: name 'train_test_split' is not defined

In [ ]:
scores = ['precision', 'recall', 'F1']

In [ ]:
for score in scores:
    print(f"Tuning hyper-parameters for {score}")
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

In [3]:
from ipywidgets import widgets
from IPython.display import Image
from IPython.display import display, HTML

In [16]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [17]:
%matplotlib notebook

import pathlib
import warnings
import pickle

from IPython.display import display, Markdown


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

from wms.dbs import pit, csv, weadbs, preferences
from wms.analysis import classification as clf

from wms.dbs.csv import nbatct as nbct
from wms.defects import asym, maxslope

Using bladecontrol plotting style.
Registering webvis-style.


c:\users\w012028\repositories\python\von_sebastianbitzer\wms-python\wms\analysis\classification.py:13: UserWarning: xgboost package could not be imported! You will only be able to preprocess data!
  warnings.warn("xgboost package could not be imported! "


# Check asymmetry for CtR sensors

In [4]:
stvars = ['azimuth_mean', 'azimuth_sigma', 'omega_mean', 'omega_sigma', 
          'pitch_mean', 'pitch_sigma', 'power_mean', 'power_sigma',
          'temperature_mean', 'wind_mean', 'wind_sigma']

In [5]:
def plot_labels(db, cycinfo, isdefect, channel, metric, y='wind_mean', ax=None):
    cycinfo = cycinfo.loc(db)
    isdefect = isdefect.loc(db)
    
    if ax is None:
        _, ax = plt.subplots()
    
    data = cycinfo.st[y] if y in cycinfo.st else cycinfo.se[(channel, metric)]
    ax = data.plot(lw=1, color='0.6', label=metric if y is None else y, ax=ax)
    
    if y is None:
        if metric == 'maxslope':
            ax.set_yscale('log')
        ax.set_ylabel(metric)
    else:
        ax.set_ylabel(y)
        
    data[isdefect.se[(channel, metric)] == 1].plot(
        ax=ax, style='.k', label='defect')
    data[isdefect.se[(channel, metric)] == 0].plot(
        ax=ax, style='.', color='tab:cyan', label='normal')
    
    ax.set_title(f'{db}\n{metric} defect labels for {channel}')
    ax.get_figure().tight_layout()
    
    return ax


def nbct(db, ct, nb=1, days=1):
    ct = pd.Timestamp(ct)
    
    if nb == 0:
        return ct
    elif nb > 0:
        cts = weadbs.cdef_query(
            db, cycle='hour', where={
                'create_time': (ct + pd.Timedelta(minutes=1), ct + pd.Timedelta(days=days)),
                'available_data': {2, 3}},
            columns=['create_time', 'ID', 'available_data'])
    else:
        nb = -nb
        cts = weadbs.cdef_query(
            db, cycle='hour', where={
                'create_time': (ct - pd.Timedelta(days=days), ct),
                'available_data': {2, 3}},
            columns=['create_time', 'ID', 'available_data']).sort_index(ascending=False)
    
    if cts.empty:
        return pd.NaT
    
    atcts = cts[cts.available_data == 3]
    if not atcts.empty:
        if nb <= atcts.shape[0]:
            return atcts.create_time.iloc[nb - 1]
        else:
            return pd.NaT
    
    nextat = pd.NaT
    for cycle in cts.itertuples():
        at = csv.load_at_dir(db, cycle.create_time, cycle.ID, oris=['edge'], blades=[1])
        if not at.empty:
            nb -= 1
            if nb == 0:
                nextat = cycle.create_time
                break
    
    return nextat


def check_at(cycinfo, db, ct, ori='edge', blades=(1, 2, 3), with_diff=True,
             labelmetrics=['maxslope', 'asym95_m0std1_split'], fig=None, axes=None):
    st = weadbs.cdef_query(db, cycle='hour', where={'create_time': ct},
                           columns=stvars + ['create_time', 'ID']).loc[0]
    
    tdata = csv.load_at_dir(db, ct, int(st.ID),
                            oris=(ori,) if isinstance(ori, str) else ori, 
                            blades=blades).astype(float)
    if tdata.empty:
        print('No AT-data!')
        return axes
    
    if cycinfo.se.index.nlevels == 2:
        cycinfo = cycinfo.loc(db)
    try:
        metrics = cycinfo.se.loc[ct]
    except KeyError:
        metrics = None
        
    stinfo = (f'\nomega = {st.omega_mean:.3f}, power = {st.power_mean:.0f}, '
              f'pitch = {st.pitch_mean:.0f}, wind = {st.wind_mean:.1f}')
    
    if axes is None:
        if fig is None:
            fig, _ = plt.subplots(
                1 + with_diff, 1, sharex=True, figsize=(7.8, 7 if with_diff else 5),
                squeeze=False)
        else:
            if fig.axes:
                for ax in fig.axes[:1 + with_diff]:
                    ax.clear()
            else:
                axes = fig.subplots(
                    1 + with_diff, 1, sharex=True, squeeze=False)
        
        axes = fig.axes
        
    ax = axes[0]
    ax = tdata.plot(lw=1, ax=ax)
    
    ax.set_title(f'{db} - {ct}{stinfo}')
    
    legend = []
    for ch in tdata.columns:
        label = ch
        if metrics is not None:
            label = f'{label}: ' + ', '.join(
                [f'{m:.2f}' for m in metrics.xs(ch)[labelmetrics]])
        legend.append(label)
    ltitle = '' if metrics is None else '\n'.join(labelmetrics)
            
    ax.set_ylabel('amplitude (digits)')
    
    if with_diff:
        ax = tdata.diff().plot(lw=1, ax=axes[1])
        ax.set_ylabel('diff (digits)')
    
    ax.legend(legend, title=ltitle)
        
    fig.tight_layout()
    
    return axes


def check_af(cycinfo, db, ct, ori='edge', blades=(1, 2, 3), fband=(2, 10), log=False,
             fig=None, ax=None):
    if cycinfo.st.index.nlevels == 2:
        cycinfo = cycinfo.loc(db)
    try:
        st = cycinfo.st.loc[ct]
    except KeyError:
        st = weadbs.cdef_query(db, cycle='hour', where={'create_time': ct},
                               columns=stvars + ['create_time', 'ID']).loc[0]
        
    stinfo = (f'\nomega = {st.omega_mean:.3f}, power = {st.power_mean:.0f}, '
              f'pitch = {st.pitch_mean:.0f}, wind = {st.wind_mean:.1f}')
    
    if ax is None:
        if fig is None:
            fig, ax = plt.subplots(constrained_layout=True)
        else:
            if fig.axes:
                ax = fig.axes[0]
                ax.clear()
            else:
                ax = fig.subplots()
    
    af = csv.load_af(db, ct, int(st.ID), chform='{blade}_{ori}',
                     oris=(ori,) if isinstance(ori, str) else ori, blades=blades)
    
    if not af.empty:
        af = af.unstack('channel').loc[slice(*fband)].sort_index(axis=1)
        af.loc[slice(*fband)].plot(ax=ax)

        ax.set_title(f'{db} - {ct}{stinfo}')
        ax.set_ylabel('amplitude (digits)')

        if log:
            ax.set_yscale('log')
            ax.set_ylim(af.stack().min(), ax.get_ylim()[1])
    
    return ax

### Load precompiled data and information

#### Precalculated metrices

In [18]:
cycpath = 'defect_metrics_acc_normal__since2014_v4.h5'

In [19]:
with pd.HDFStore(cycpath, 'r') as store:
    _, ctrdbs, _ = next(store.walk())
    ctrdbs = pd.Index(ctrdbs, name='dbname')
    
options = pd.read_hdf(cycpath, 'options').to_dict()
where_data = eval(pd.read_hdf(cycpath, 'where')['where'])
    
cycinfo = weadbs.SEData.concat(
    [weadbs.SEData.from_hdf(cycpath, prefix=db) for db in ctrdbs],
    keys=ctrdbs)

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [ ]:
cycinfo

#### Infos about turbines and tickets

In [ ]:
df_tickets = pd.read_pickle("df_tickets.pkl")
df_tickets

#### Load turbine infos of turbines contained in precalculated metrics data

In [ ]:
weainfo = pit.query_pit(
    f"""SELECT Datenbankname, WEA_Typ#Name, WEA_Name, Windpark_WEA#Windparkname
        FROM VIEW_Windkraftanlagen 
        WHERE Datenbankname in ({', '.join(['%s'] * ctrdbs.size)})""",
    tuple(ctrdbs)).set_index('Datenbankname')

weainfo['N'] = cycinfo.se.groupby(level='dbname').size()
weainfo

### Display infos about precalculated metrices

In [ ]:
data = cycinfo.se
#data = cycinfo.select_st().se.stack('channel')
data.tail()

In [ ]:
# function to label the time series of df on behalf of the defect periods given in dict_tickets, it returns the indices of that data sets that are within the sensor defect times
def label_metrics_data(cycinfo, df_tickets):
    #sfmt = '%Y-%m-%d %H:%M:%S'
    res = dict()
    res_ix_list = list()
    for idx, row in df_tickets.iterrows():
        db = row['db']
        ticket_id = row['ticket_id']
        defect_times = row['defect_times']  
        
        data = cycinfo.loc(db).se.drop_duplicates().index
        
        for ch, pers in defect_times.items():
            ixs = list()
            for per in pers:
                if len(per)==1:
                    cts = (data[data >= per[0]])
                else:
                    cts = (data[(data >= per[0]) & (data<per[1])])
                ixs.append(cts)
                [res_ix_list.append((db, ct, ch)) for ct in cts]
            res.update({(db, ch): ixs})
            
    res_ix = pd.MultiIndex.from_tuples(res_ix_list, names=['dbname', 'create_time', 'channel'])
    return res, res_ix

In [ ]:
defect_times_by_type = dict()
for g, dfg in df_tickets.groupby('defecttype'):
    _ddt, idt = label_metrics_data(cycinfo, dfg)
    defect_times_by_type.update({g: idt})

In [ ]:
dict_defect_times, idx_defect_times = label_metrics_data(cycinfo, df_tickets)
#dict_defect_times

In [ ]:
dict_defect_times

## Histograms

### By sensor orientation and operational state

In [ ]:
# only start at 3 m/s wind, because we anyway won't be able 
# to detect ice for lower wind
# exclude trudeln for now by selecting high pitch
where_stand = weadbs.Where(
    {'omega_mean': 0, 
     'power_mean': (-500, 1), 
     'wind_mean': (3,),
     'pitch_mean': (40,)})

where_betrieb = weadbs.Where(
    {'omega_mean': (0.05,),
     'power_mean': (1,),
     'wind_mean': (3,)})

def make_bins(opts):
    
    if 'xmax' in opts:
        # original version
        #opts.setdefault(
        #    'bins', np.r_[np.linspace(0, opts['xmax'], opts['B']), 10000])
        
        # modified by CK, otherwise error occurs if xmax>10000
        tmp = opts['xmax']
        if tmp<10000:
            u = 10000
        else:
            u = np.power(10, int(np.ceil(np.log10(tmp))+2))
        opts.setdefault(
            'bins', np.r_[np.linspace(0, opts['xmax'], opts['B']), u]) 
        opts.setdefault('xlim', opts['xmax'] * 1.1)
    else:
        opts.setdefault('bins', opts['B'])

def metric_hist(cycinfo, metric, standopts={}, betriebopts={}, isdefect=None, figsize=(7.8, 6)):
    standopts = standopts.copy()
    standopts.setdefault('where', where_stand)
    standopts.setdefault('B', 100)
    standopts.setdefault('maxslopethr', 100)
    make_bins(standopts)
    
    betriebopts = betriebopts.copy()
    betriebopts.setdefault('where', where_betrieb)
    betriebopts.setdefault('B', 100)
    betriebopts.setdefault('maxslopethr', 800)
    make_bins(betriebopts)
    
    fig, axes = plt.subplots(
        2, 2, sharex='col', sharey='col', constrained_layout=True, figsize=figsize)
    
    for ori, row in zip(['edge', 'flap'], axes):
        data = cycinfo.select_st(standopts['where'], ori).se.stack('channel')
        if isdefect is not None:
            ind = isdefect.select_st(standopts['where'], ori).se.stack(
                'channel')['true'].astype(bool)
        elif 'maxslopethr' in standopts:
            ind = data['maxslope'] > standopts['maxslopethr']
        else:
            ind = None
        
        if ind is not None:
            ax = sns.distplot(
                data.loc[ind, metric], kde=False, bins=standopts['bins'],
                ax=row[0], label='defect')
            ax = sns.distplot(
                data.loc[~ind, metric], kde=False, bins=standopts['bins'],
                ax=row[0], label='normal')
            
            ax.legend(title='maxslope' if isdefect is None else 'true label')
        else:
            ax = sns.distplot(
                data[metric], kde=False, bins=standopts['bins'], ax=row[0])
        
        ax.set_ylabel(f'{ori}\ncycle count')
        
        data = cycinfo.select_st(betriebopts['where'], ori).se.stack('channel')
        if isdefect is not None:
            ind = isdefect.select_st(betriebopts['where'], ori).se.stack(
                'channel')['true'].astype(bool)
        elif 'maxslopethr' in betriebopts:
            ind = data['maxslope'] > betriebopts['maxslopethr']
        else:
            ind = None
        
        if ind is not None:
            ax = sns.distplot(
                data.loc[ind, metric], kde=False, bins=betriebopts['bins'],
                ax=row[1], label='defect')
            ax = sns.distplot(
                data.loc[~ind, metric], kde=False, bins=betriebopts['bins'],
                ax=row[1], label='normal')
            
            ax.legend(title='maxslope' if isdefect is None else 'true label')
        else:    
            ax = sns.distplot(
                data[metric], kde=False, bins=betriebopts['bins'], ax=row[1])
        
    for ax in axes[0, :]:
        ax.set_xlabel('')
        
    axes[0, 0].set_title('stand')
    axes[0, 1].set_title('betrieb')
    if 'xlim' in standopts:
        axes[0, 0].set_xlim(0, standopts['xlim'])
    if 'xlim' in betriebopts:
        axes[0, 1].set_xlim(0, betriebopts['xlim'])
    
    return axes


# new variant 2021-3-18, defect/ok as parameter, not determined by max_slope
def metric_hist2(cycinfo, metric, idx_defect_times, standopts={}, betriebopts={}, figsize=(7.8, 6)):
    standopts = standopts.copy()
    standopts.setdefault('where', where_stand)
    #standopts.setdefault('B', 100)
    #standopts.setdefault('maxslopethr', 100)
    make_bins(standopts)
    
    betriebopts = betriebopts.copy()
    betriebopts.setdefault('where', where_betrieb)
    #betriebopts.setdefault('B', 100)
    #betriebopts.setdefault('maxslopethr', 800)
    make_bins(betriebopts)
    
    fig, axes = plt.subplots(
        2, 2, sharex='col', sharey='col', constrained_layout=True, figsize=figsize)
    
    for ori, row in zip(['edge', 'flap'], axes):
        data = cycinfo.select_st(standopts['where'], ori).se.stack('channel').drop_duplicates()
                
        if idx_defect_times is not None:
            ind = data.index.intersection(idx_defect_times)
            not_ind = data.index.difference(idx_defect_times)
            ax = sns.distplot(
                data.loc[ind, metric], kde=False, bins=standopts['bins'],
                ax=row[0], label='defect')
            ax = sns.distplot(
                data.loc[not_ind, metric], kde=False, bins=standopts['bins'],
                ax=row[0], label='normal')
            
            ax.legend(title='true label')
        else:
            ax = sns.distplot(
                data[metric], kde=False, bins=standopts['bins'], ax=row[0])
        
        ax.set_ylabel(f'{ori}\ncycle count')
        
        data = cycinfo.select_st(betriebopts['where'], ori).se.stack('channel').drop_duplicates()
        
        if idx_defect_times is not None:
            ind = data.index.intersection(idx_defect_times)
            not_ind = data.index.difference(idx_defect_times)
            ax = sns.distplot(
                data.loc[ind, metric], kde=False, bins=betriebopts['bins'],
                ax=row[1], label='defect')
            ax = sns.distplot(
                data.loc[not_ind, metric], kde=False, bins=betriebopts['bins'],
                ax=row[1], label='normal')
            
            ax.legend(title='true label')
        else:    
            ax = sns.distplot(
                data[metric], kde=False, bins=betriebopts['bins'], ax=row[1])
        
    for ax in axes[0, :]:
        ax.set_xlabel('')
        
    axes[0, 0].set_title('stand')
    axes[0, 1].set_title('betrieb')
    if 'xlim' in standopts:
        axes[0, 0].set_xlim(0, standopts['xlim'])
    if 'xlim' in betriebopts:
        axes[0, 1].set_xlim(0, betriebopts['xlim'])
    
    return axes



# new variant 2021-3-27, defect/ok as parameter, not determined by max_slope
def metric_hist3(cycinfo, metric, defect_times_by_type, standopts={}, betriebopts={}, figsize=(7.8, 6)):
    standopts = standopts.copy()
    standopts.setdefault('where', where_stand)
    #standopts.setdefault('B', 100)
    #standopts.setdefault('maxslopethr', 100)
    make_bins(standopts)
    
    betriebopts = betriebopts.copy()
    betriebopts.setdefault('where', where_betrieb)
    #betriebopts.setdefault('B', 100)
    #betriebopts.setdefault('maxslopethr', 800)
    make_bins(betriebopts)
    
    fig, axes = plt.subplots(
        2, 2, sharex='col', sharey='col', constrained_layout=True, figsize=figsize)
    
    for ori, row in zip(['edge', 'flap'], axes):
        data = cycinfo.select_st(standopts['where'], ori).se.stack('channel').drop_duplicates()
        not_ind = data.index
        for deftype, idx_defect_times in defect_times_by_type.items():
            ind = data.index.intersection(idx_defect_times)
            #not_ind = data.index.difference(idx_defect_times)
            not_ind = not_ind.difference(idx_defect_times)
            ax = sns.distplot(data.loc[ind, metric], kde=False, bins=standopts['bins'], ax=row[0], label=defect_type)
        ax = sns.distplot(data.loc[not_ind, metric], kde=False, bins=standopts['bins'], ax=row[0], label='normal')
            
        ax.legend(title='true label')
        #else:
        #    ax = sns.distplot(
        #        data[metric], kde=False, bins=standopts['bins'], ax=row[0])
        
        ax.set_ylabel(f'{ori}\ncycle count')
        
        data = cycinfo.select_st(betriebopts['where'], ori).se.stack('channel').drop_duplicates()
        not_ind = data.index
        for def_type, idx_defect_times in def_times_by_type.items():
            ind = data.index.intersection(idx_defect_times)
            #not_ind = data.index.difference(idx_defect_times)
            not_ind = not_ind.difference(idx_defect_times)
            ax = sns.distplot(data.loc[ind, metric], kde=False, bins=betriebopts['bins'], ax=row[1], label=def_type)
        ax = sns.distplot(data.loc[not_ind, metric], kde=False, bins=betriebopts['bins'], ax=row[1], label='normal')
        ax.legend(title='true label')
        
    for ax in axes[0, :]:
        ax.set_xlabel('')
        
    axes[0, 0].set_title('stand')
    axes[0, 1].set_title('betrieb')
    if 'xlim' in standopts:
        axes[0, 0].set_xlim(0, standopts['xlim'])
    if 'xlim' in betriebopts:
        axes[0, 1].set_xlim(0, betriebopts['xlim'])
    
    return axes



# create boxplots for all defect types
def metric_box(cycinfo, metric, defect_times_by_type, standopts={}, betriebopts={}, figsize=(7.8, 6)):
    standopts = standopts.copy()
    standopts.setdefault('where', where_stand)
    #standopts.setdefault('B', 100)
    #standopts.setdefault('maxslopethr', 100)
    make_bins(standopts)
    
    betriebopts = betriebopts.copy()
    betriebopts.setdefault('where', where_betrieb)
    #betriebopts.setdefault('B', 100)
    #betriebopts.setdefault('maxslopethr', 800)
    make_bins(betriebopts)
    
    fig, axes = plt.subplots(
        2, 2, sharex='col', sharey='col', constrained_layout=True, figsize=figsize)
    
    for ori, row in zip(['edge', 'flap'], axes):
        data = cycinfo.select_st(standopts['where'], ori).se.stack('channel').drop_duplicates().assign(deftype='ok')
        for deftype, idx_defect_times in defect_times_by_type.items():
            ind = data.index.intersection(idx_defect_times)
            data.loc[ind, 'deftype'] = deftype
            
        sns.boxplot(data=data, x='deftype', y=metric, ax=ax)
        ax.set_ylabel(f'{ori}\ncycle count')
        
        data = cycinfo.select_st(betriebopts['where'], ori).se.stack('channel').drop_duplicates().assign(deftype='ok')
        for def_type, idx_defect_times in def_times_by_type.items():
            ind = data.index.intersection(idx_defect_times)
            data.loc[ind, 'deftype'] = deftype
            
        sns.boxplot(data=data, x='deftype', y=metric, ax=ax)
        ax.set_ylabel(f'{ori}\ncycle count')
        
    for ax in axes[0, :]:
        ax.set_xlabel('')
        
    axes[0, 0].set_title('stand')
    axes[0, 1].set_title('betrieb')
    if 'xlim' in standopts:
        axes[0, 0].set_xlim(0, standopts['xlim'])
    if 'xlim' in betriebopts:
        axes[0, 1].set_xlim(0, betriebopts['xlim'])
    
    return axes

In [ ]:
histopts = {
    'maxslope': {'standopts': {'xmax': 33000}, 'betriebopts': {'xmax': 33000}},
    #'asym95_std1': {'standopts': {'xmax': 6}, 'betriebopts': {'xmax': 6}},
    #'asym95_m0std1': {'standopts': {'xmax': 6}, 'betriebopts': {'xmax': 6}},
    'asym95_m0std1_split': {'standopts': {'xmax': 3.5}, 'betriebopts': {'xmax': 3.5}},
    'max_amp': {'standopts': {'xmax': 12000}, 'betriebopts': {'xmax': 33000}}#,
    #'se': {'standopts': {'xmax': 1000}, 'betriebopts': {'xmax': 3000}}
}

In [ ]:
for _metric, _opts in histopts.items():
    display(Markdown('### ' + _metric))
    #axes = metric_hist2(cycinfo, _metric, idx_defect_times, **_opts)
    axes = metric_hist3(cycinfo, _metric, defect_times_by_type, **_opts)

## By turbine and channel

In [ ]:
chs_edge = ['101_edge', '102_edge', '103_edge']
chs_flap = ['101_flap', '102_flap', '103_flap']
chs_all = np.r_[chs_edge, chs_flap]

In [ ]:
def plot_hists_by_db(df_tickets, cycinfo, metric, opwhere=dict(), chs = chs_all, bins=np.r_[np.linspace(0,30000,100), 40000], xlim = [-10, 20000]):

    ctrdbs = df_tickets.db.unique()
    cp = sns.color_palette("Paired")
    cp2 = [cp[i] for i in [5,4,1,0,3,2]]    
    
    fig, axes = plt.subplots(ctrdbs.size, 1, sharex=True, constrained_layout=True,
                             figsize=(9.3, max(4, 2 * ctrdbs.size)))

    for ax, _db in zip(axes, ctrdbs):
        df = df_tickets[df_tickets.db==_db].loc[:, ['defect_times', 'defecttype']]
        for _idx, row in df.iterrows():
            sinfo = ",".join(row['defect_times'].keys()) + f": {row['defecttype']}"
        
        _data = cycinfo.select_st(opwhere).se.loc[_db].xs(metric, axis=1, level=1, drop_level=True).drop_duplicates()
        if not _data.empty:
            sns.histplot(data=_data, ax=ax, kde=True, legend=True, bins=bins, palette = cp2)
            #sns.displot(data=_data, ax=ax, kde=False)#, bins=np.r_[np.linspace(0, 1000, 100), 100000])#, hue=, row=None, col=None, weights=None, kind='hist', rug=False, rug_kws=None, log_scale=None, legend=True, palette=None, hue_order=None, hue_norm=None, color=None, col_wrap=None, row_order=None, col_order=None, height=5, aspect=1, facet_kws=None, **kwargs)¶
#                             bins=np.r_[np.linspace(0, 1000, 100), 10000])

        #ax.legend()
        ax.set_ylabel(_db)
        ax.set_xlabel('')
        ax.set_xlim(xlim)
        ax.set_title(sinfo, y=0.85, fontsize=10)

### max-slope

In [ ]:
plot_hists_by_db(df_tickets, cycinfo, 'maxslope')

In [ ]:
weitere Darstellungen anfertigen:
    - aufgespalten nach stand/betrieb
    - aufgespalten nach edge/flap 
    - aufgespalten nach defekt/ok

Dabei herausfinden:
    - was sind sinnvolle Schwellwerte fuer Stand und Betrieb, edge und flap
    - uebriggebliebene Defekte nach den jeweils anderen Metriken darstellen

### max amplitude

In [ ]:
plot_hists_by_db(df_tickets, cycinfo, 'max_amp', bins=np.r_[np.linspace(0,3000,100), 4000], xlim=[-10, 2000])

### asymmetry

In [ ]:
plot_hists_by_db(df_tickets, cycinfo, 'asym95_m0std1_split', bins=np.arange(0, 3.5, 0.05), xlim = [0, 3.5])

TODO:

* inkonsistent aussehende Kanaele durchgehen und weglassen, re-labeln o.ae.
* jeweils 1 Diagramm fuer (stand,betrieb) x (edge,flap) x metrics anfertigen:
    
    * Boxplots fuer ok, kurzschluss, offener kanal usw. nebeneinander darstellen
    * Histogramme fuer ok, Kurzschluss, offener Kanal usw. nebeneinander darstellen
    
* aus unabhaengig voneinander bestimmten Werte F1, false positives etc. fuer jede einzelne Defektform und insgesamt berechnen und darstellen
* pipeline bauen, Parameter in Kombination ermitteln fuer jede WEA einzeln und insgesamt
* mit unabhaengig voneinander ermittelten Parametern vgl.n nebeneinander
* false positives/true negatives getrennt betrachten

In [ ]:
# TODO 2021-3-23: alle Daten (se, st, tickest) kombinieren um einen Dataframe mit Betriebszustaenden, Metriken und Defekttyp zu erzeugen fuer einfachere Erstellung von Boxplots etc. Dazu se wide to long, mit st joinen, durch df_tickets durchgehen und ok/Defekttyp eintragen    

In [ ]:
df_tickets.head(2)

In [ ]:
df_se = cycinfo.se.drop_duplicates().reset_index().head()
print(df_se)
pd.melt(df_se, id_vars='date', value_vars=['AA', 'BB', 'CC'])

In [ ]:
cycinfo.st.head(2)

In [ ]:
df = cycinfo.se.drop_duplicates().merge(cycinfo.st.drop_duplicates(), left_index=True, right_index=True)

In [ ]:
cycinfo.st.tail()

In [ ]:
df.head()

## Quantile-based thresholds and relation between metrics
### q-values corresponding to maxslope thresholds

In [ ]:
maxslope_thresh = {'stand': 100, 'betrieb': 800}

In [ ]:
for opstate, opwhere in zip(['stand', 'betrieb'], [where_stand, where_betrieb]):
    _qval = (cycinfo.select_st(opwhere).se.stack('channel')['maxslope'] 
             < maxslope_thresh[opstate]).mean()
    print(f'{opstate}: {_qval:.3f}')

### 95% quantile thresholds

In [ ]:
thresholds = pd.DataFrame(
    dtype=float, columns=pd.Index(['stand', 'betrieb'], name='opstate'),
    index=cycinfo.se.columns.levels[1].set_names('metric'))

isdefect = weadbs.SEData(cycinfo.se.applymap(lambda v: -1), cycinfo.st)

for opstate, opwhere in zip(thresholds.columns, (where_stand, where_betrieb)):
    _metrics = cycinfo.select_st(opwhere)
    thresholds.loc[:, opstate] = _metrics.se.stack('channel').quantile(0.95)
    
    # overwrite maxslope thresholds with agreed upon values
    thresholds.loc['maxslope', opstate] = maxslope_thresh[opstate]
    
    # overwrite asym95_m0std1_split thresholds with values giving low FPs
    thresholds.loc['asym95_m0std1_split', opstate] = 1.1 if opstate == 'betrieb' else 0.7
    
    isdefect.se.loc[_metrics.se.index, :] = cycinfo.se.loc[_metrics.se.index, :].apply(
        lambda s: s > thresholds.loc[s.name[1], opstate]).astype(int)

In [ ]:
thresholds

### Correlations between classifications based on the different metrics

In [ ]:
corrs = isdefect.se.stack('channel').corr()

In [ ]:
corrs

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 3))

img = ax.matshow(corrs)
ax.grid(False)
ax.set_yticks(np.arange(corrs.shape[0]))
ax.set_yticklabels(corrs.index)
ax.set_xticklabels([]);

cb = fig.colorbar(img, ax=ax)

### Overall fraction of cycles identified by maxslope that are also identified by the other metric

In [ ]:
isdefect_stack = isdefect.se.stack('channel')

In [ ]:
isdefect_stack.agg(lambda s: (s[isdefect_stack.maxslope == 1] == 1).mean())

## Lametta example
- note: from cycle 2020-08-03 13:04:00 to cycle 2020-08-04 08:34:33 (both inclusive) either omega_mean has None, or, even more odd, the AT-data stored for this turbine is obviously from an acceleration sensor and not from a strain sensor - so this is mixed up!

In [ ]:
db = 'cmrblba_bc_t_02395_strain'

In [ ]:
axes = cycinfo.loc(db).plot_se('maxslope')

In [ ]:
axes = cycinfo.loc(db).plot_se('asym95_m0std1_split')

In [ ]:
_isd = isdefect.loc(isdefect.se.iloc[:, 0] >= 0).loc(db).se

ytrue = _isd.applymap(lambda v: 0)
ytrue['102_edge'] = 1
ytrue['102_flap'] = 1

_isd = _isd.stack('channel')
ytrue = ytrue.stack('channel')

In [ ]:
scores = pd.concat(
    [clf.BinScores(ytrue[metric], _isd[metric]).to_series()
     for metric in ytrue.columns], axis=1, keys=ytrue.columns)

In [ ]:
scores

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax = scores.loc['F1'].plot.bar(ax=ax)

## Hard example
- in all cycles before 2020-04 blade 3, flap has a flat signal with small jumps / spikes up and down from time to time; maxslope only picks up on these jumps, when they pass threshold; asym95_m0std1_split recognises all of these cycles as defects, because the short jumps (in time) lead to an asymmetric raw AT signal; asym with amax1, or skew don't recognise these cycles consistently either

In [ ]:
db = 'cmrblba_vid_v231228'

In [ ]:
defect_period = slice(None, '2020-07-25 13:12:54')

In [ ]:
axes = cycinfo.loc(db).loc(defect_period).plot_se('maxslope')
axes[1].set_yscale('log')

In [ ]:
axes = cycinfo.loc(db).loc(defect_period).plot_se('asym95_m0std1_split')

In [ ]:
ytest = isdefect.loc(isdefect.se.iloc[:, 0] >= 0).se.loc[
    (db, defect_period), :]

ytrue = ytest.xs('maxslope', axis=1, level=1).applymap(lambda v: 0)
ytrue['103_flap'] = 1

ytest = ytest.stack('channel')
ytrue = ytrue.stack('channel')

In [ ]:
scores = pd.concat(
    [clf.BinScores(ytrue, ytest[metric]).to_series()
     for metric in ytest.columns], axis=1, keys=ytest.columns)

In [ ]:
scores

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax = scores.loc['F1'].plot.bar(ax=ax)
ax.set_ylabel('F1');

### Comparison to maxslope

In [ ]:
maxslope_FN = ytest[(ytrue == 1) & (ytest.maxslope == 0)]
maxslope_TN = ytest[(ytrue == 0) & (ytest.maxslope == 0)]

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, constrained_layout=True, figsize=(5, 5))

ax = (maxslope_FN).sum().plot.bar(ax=axes[0])
ax.set_ylabel('# corrected FN')
ax = (maxslope_TN).sum().plot.bar(ax=axes[1])
ax.set_ylabel('# new FP');

### Combination with maxslope

In [ ]:
ytest_combined = ytest.apply(lambda s: s | ytest.maxslope)

In [ ]:
scores_combined = pd.concat(
    [clf.BinScores(ytrue, ytest_combined[metric]).to_series()
     for metric in ytest.columns], axis=1, keys=ytest_combined.columns)

In [ ]:
scores_combined

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax = scores_combined.loc['F1'].plot.bar(ax=ax)
ax.set_ylabel('F1');
ax.set_ylim(0.5, 0.75);

In [ ]:
_ind = ytrue[ytest_combined['asym95_m0std1_split'] != ytrue].unstack('channel')['103_flap'].dropna().index.get_level_values('create_time')

In [ ]:
_ori = 'flap'
_ct = nbct(db, '2020-07-24 18:03:12', nb=0, days=12)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)
cycinfo.se.loc[(db, _ct), '103_flap']

In [ ]:
def score_metric(ytrue, cycinfo, metric, threshs, opstate='betrieb', ybase=None):
    ytrue = ytrue.unstack('channel')
    
    values = cycinfo.loc(ytrue.index).select_st(
        where_betrieb if opstate == 'betrieb' else where_stand).se.xs(
            metric, axis=1, level=1)
    ytrue = ytrue.loc[values.index].stack('channel')
    
    if ybase is not None:
        ybase = ybase.unstack('channel').loc[values.index].stack('channel')
    
    scores = []
    for thr in threshs:
        ytest = (values > thr).stack()
        if ybase is not None:
            ytest = ytest | ybase.astype(bool)
            
        _scores = clf.BinScores(ytrue, ytest.astype(int)).to_series()
        _scores['FP'] = _scores['FPR'] * (ytrue == 0).sum()
        _scores['TP'] = _scores['recall'] * (ytrue == 1).sum()
        
        scores.append(_scores)
        
    return pd.concat(scores, keys=threshs, axis=1).T, ytrue.size

In [ ]:
pd.Series(_ind).to_csv(f'{db}_103_flap_undetected_defects.csv')

In [ ]:
_metric = 'asym95_m0std1_split'
_base_metric = 'maxslope'
_thrs = np.r_[np.arange(0, 2.01, 0.02), np.arange(2.5, 3.5, 0.5)]

scores_betrieb_base, N_betrieb = score_metric(
    ytrue, cycinfo, _base_metric, [thresholds.loc[_base_metric, 'betrieb']])
scores_stand_base, N_stand = score_metric(
    ytrue, cycinfo, _base_metric, [thresholds.loc[_base_metric, 'stand']], 'stand')

scores_betrieb, N_betrieb = score_metric(
    ytrue, cycinfo, _metric, _thrs, ybase=ytest[_base_metric])
scores_stand, N_stand = score_metric(
    ytrue, cycinfo, _metric, _thrs, 'stand', ybase=ytest[_base_metric])

In [ ]:
db

In [ ]:
N_stand

In [ ]:
scores_betrieb_base['recall']

In [ ]:
fig, ax = plt.subplots()

(scores_stand['TP'] - scores_stand_base['TP'].iloc[0]).plot(
    ax=ax, label='additional TP')
(scores_stand['FP'] - scores_stand_base['FP'].iloc[0]).plot(
    ax=ax, label='additional FP')

ax.set_title('stand')
ax.set_yscale('log')
ax.legend()
ax.set_xlabel('threshold')
ax.set_ylabel('# cycles');

In [ ]:
fig, ax = plt.subplots()

(scores_betrieb['TP'] - scores_betrieb_base['TP'].iloc[0]).plot(
    ax=ax, label='additional TP')
(scores_betrieb['FP'] - scores_betrieb_base['FP'].iloc[0]).plot(
    ax=ax, label='additional FP')

ax.set_title('betrieb')
ax.set_yscale('log')
ax.legend()
ax.set_xlabel('threshold')
ax.set_ylabel('# cycles');

In [ ]:
ax = plot_labels(db, cycinfo, isdefect, '103_flap', 'maxslope', y=None)

In [ ]:
ax = plot_labels(db, cycinfo, isdefect, '103_flap', 'asym95_m0std1_split', y=None)

In [ ]:
_ori = 'flap'
_ct = nbct(db, '2020-02-27 22:56:52', nb=0, days=12)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)

In [ ]:
_isd = ytest.loc[db].xs('103_flap', level='channel')

In [ ]:
(_isd[(_isd.maxslope == 0) & (_isd.asym95_m0std1_split == 0)]).index

In [ ]:
_ori = 'flap'
_ct = nbct(db, '2020-06-03 20:00:21', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)
isdefect.loc(db).se.loc[_ct, '103_flap']

In [ ]:
store.keys()

### False positives
#### 102_flap
- large, asymmetric amplitudes (periodic, thin peaks) in AT
- unclear asymmetry in example in stand - threshold barely crossed

In [ ]:
ch = '102_flap'

In [ ]:
_isd = ytest.loc[db].xs(ch, level='channel')

In [ ]:
(_isd[(_isd.maxslope == 1) | (_isd.asym95_m0std1_split == 1)]).index

In [ ]:
_ori = ch[-4:]
_ct = nbct(db, '2020-07-22 01:06:31', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori, blades=(1, 2))
axaf = check_af(cycinfo, db, _ct, _ori, blades=(1, 2))
isdefect.loc(db).se.loc[_ct, ch]

In [ ]:
_ori = ch[-4:]
_ct = nbct(db, '2020-06-23 04:02:04', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori, blades=(1, 2))
axaf = check_af(cycinfo, db, _ct, _ori, blades=(1, 2))
isdefect.loc(db).se.loc[_ct, ch]

#### 101_flap
- asymmetric, thin peaks in AT, close to threshold
- unclear asymmetry in stand, close to threshold

In [ ]:
ch = '101_flap'

In [ ]:
_isd = ytest.loc[db].xs(ch, level='channel')

In [ ]:
(_isd[(_isd.maxslope == 1) | (_isd.asym95_m0std1_split == 1)]).index

In [ ]:
_ori = ch[-4:]
_ct = nbct(db, '2020-06-16 22:03:56', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori, blades=(1, 2))
axaf = check_af(cycinfo, db, _ct, _ori, blades=(1, 2))
isdefect.loc(db).se.loc[_ct, ch]

#### 101_edge
- larger negative amplitudes than positive amplitudes within normal period - only locally in splits; often close to threshold
- unclear asymmetry in stand, close to threshold

In [ ]:
ch = '101_edge'

In [ ]:
_isd = ytest.loc[db].xs(ch, level='channel')

In [ ]:
(_isd[(_isd.maxslope == 1) | (_isd.asym95_m0std1_split == 1)]).index

In [ ]:
_ori = ch[-4:]
_ct = nbct(db, '2020-07-24 05:14:16', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)
isdefect.loc(db).se.loc[_ct, ch]

In [ ]:
_ori = ch[-4:]
_ct = nbct(db, '2020-06-18 14:12:21', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)
isdefect.loc(db).se.loc[_ct, ch]

### Distribution of metrics over true positives and negatives

In [ ]:
_labels = pd.DataFrame({'true': ytrue}).unstack('channel').reorder_levels([1, 0], axis=1)
truedefect = weadbs.SEData(_labels, cycinfo.st.loc[_labels.index])

In [ ]:
_metric = 'maxslope'

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    metric_hist(cycinfo.loc(truedefect.st.index), _metric, isdefect=truedefect,
                **histopts[_metric])

In [ ]:
_metric = 'asym95_m0std1_split'

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    metric_hist(cycinfo.loc(truedefect.st.index), _metric, isdefect=truedefect,
                **histopts[_metric])

## Siemens Gamesa 1
- blade 1, edge: one cycle at 2020-07-25 23:06:46 with partially missing at data leading to large jump when data starts - could be classified as defect, but is single case
- blade 2, edge: low amplitude across whole time period
    - potentially low-pass behaviour
    - no clear defect - hard to detect
- blade 2, flap: "einfach defekt" across whole time period
    - mostly flat signal
    - until about 2020-06-28 high-frequency oscillations appear to be normal, but low frequencies completely suppressed
    - then until about 2020-07-20 14:10 high-frequency oscillations still appear to happen concurrently with other channels, but with much reduced amplitude
    - from 2020-07-20 17:22:49 random, extreme jumps in AT signal, but only in betrieb
    - should be detected by flat line detector

In [ ]:
db = 'cmrblba_bc_t_03078'

In [ ]:
axes = cycinfo.loc(db).plot_se('maxslope')
axes[1].set_yscale('log')

In [ ]:
axes = cycinfo.loc(db).plot_se('asym95_m0std1_split')

In [ ]:
axes = cycinfo.loc(db).plot_se('std')

In [ ]:
cycinfo.loc(db).st.loc['2020-07-20 23'].index[:10]

In [ ]:
_ori = 'edge'
_ct = nbct(db, '2020-07-25 23:06:46', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)

In [ ]:
ytest = isdefect.loc(isdefect.se.iloc[:, 0] >= 0).se.loc[db, :]

ytrue = ytest.xs('maxslope', axis=1, level=1).applymap(lambda v: 0)
#ytrue['102_edge'] = 1
ytrue['102_flap'] = 1

ytest = ytest.stack('channel')
ytrue = ytrue.stack('channel')

In [ ]:
scores = pd.concat(
    [clf.BinScores(ytrue, ytest[metric]).to_series()
     for metric in ytest.columns], axis=1, keys=ytest.columns)

In [ ]:
scores

## cmrblba_bc_t_02848

In [ ]:
db = 'cmrblba_bc_t_02848'

In [ ]:
axes = cycinfo.loc(db).plot_se('maxslope')
axes[0].set_yscale('log');

In [ ]:
axes = cycinfo.loc(db).plot_se('asym95_m0std1_split')

In [ ]:
cycinfo.loc(db).loc('2020-06-27')

In [ ]:
_ori = 'edge'
_ct = nbct(db, '2020-06-27 18:29:30', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)

## False positives in V112 at max rotor speed

In [ ]:
db = 'cmrblba_bc_t_01158'

In [ ]:
axes = cycinfo.loc(db).plot_se('maxslope', by='wind_mean')

FP rates **at max rotor speed**:

In [ ]:
isdefect.loc(db).select_st({'omega_mean': (0.205,)}).se.stack('channel').mean()

FP rates **below max rotor speed**

In [ ]:
isdefect.loc(db).select_st({'omega_mean': (0, 0.205)}).se.stack('channel').mean()

## cmrblba_bc_t_01763_strain
- RBL 3 flap ztw BCE101 einfach defekt, from 2020-06-18 - rare? as not in 300 sample
- RBL 2 edge BCE101 zu geringer Signalpegel
- RBL 2 flap BCE101 zu geringer Signalpegel, supposedly fixed on 2019-05-29, but still there

In [ ]:
db = 'cmrblba_bc_t_01763_strain'

In [ ]:
axes = cycinfo.loc(db).plot_se('maxslope')
axes[0].set_yscale('log');

In [ ]:
axes = cycinfo.loc(db).plot_se('asym95_m0std1_split')

In [ ]:
_ori = 'flap'
_ct = nbct(db, '2020-06-20 19:44:16', nb=0, days=2)
axesat = check_at(cycinfo, db, _ct, _ori)
axaf = check_af(cycinfo, db, _ct, _ori)

In [ ]:
weadbs.cdef_query(db, cycle='hour', columns=['ID'], where={'create_time': _ct}).loc[0, 'ID']

In [ ]:
tdata = csv.load_at_dir(db, _ct, 58965, oris=('edge', 'flap'))

In [ ]:
asym(tdata.astype(float), 0.95, norm='m0std1', split=3000)